# Library

In [1]:
# 標準ライブラリのインポート
import os
import json

# サードパーティライブラリのインポート
import requests
from dotenv import load_dotenv
import pandas_datareader.data as web 

# LLM関連のインポート
from openai import OpenAI

# APIキーの設定
load_dotenv() 
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

# GPTモデルの設定
model_name='gpt-4o-mini-2024-07-18' #'gpt-4o-2024-08-06'

# 株価を取得する関数

In [2]:
# DataReaderでstooqから株価情報を取得する関数を定義（開始日～終了日）
def get_stock_price(code, date_from ,date_to):
    # 日本用のコードに変換
    code = code + '.JP'
    # stooqからデータ取得
    df = web.DataReader(code, data_source='stooq', start=date_from,end=date_to)
    # 日付と株価の終値を返す ※日付形式がおかしくなるのでorient='table'を利用
    return df['Close'].to_json(orient='table')

def get_stock_price_range(arguments):
    res = get_stock_price(
        code = arguments.get('code'),
        date_from = arguments.get('date_from'),
        date_to = arguments.get('date_to')
    )
    return res

# 天気を取得する関数

In [3]:
# 天気情報を取得する関数を定義
def get_info(latitude, longitude):
    url = "https://api.open-meteo.com/v1/forecast"
    parameters = {
        "latitude": latitude,
        "longitude": longitude,
        "current_weather": "true"}
    response = requests.get(url, params=parameters)

    if response.status_code == 200:
        data = response.json()
        return json.dumps(data["current_weather"])
    else:
        return None

def get_weather_by_location(arguments):
    res = get_info(
        # 緯度と経度の情報 取得
        latitude = arguments.get('latitude'),
        longitude = arguments.get('longitude')
    )
    return res

# Function callingの設定

In [4]:
# 使用するツールのリスト
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_stock_price_range",  
            "description": "証券コードと2つの日付を渡すと指定の会社のその2つの日付の間の株価の終値を返します",  
            "parameters": {
                "type": "object",
                "properties": {
                    "code": {
                        "type": "string",
                        "description": "証券コード",
                    },
                    "date_from": {
                        "type": "string",
                        "description": "株価を知りたい日付の開始日",
                    },
                    "date_to": {
                        "type": "string",
                        "description": "株価を知りたい日付の終了日",
                    },
                },
                "required": ["code", "date_from", "date_to"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_weather_by_location",  
            "description": "緯度と経度の情報から現在と将来の天気を取得",
            "parameters": {
                "type": "object",
                "properties": {
                    "latitude": {
                        "type": "string",
                        "description": "緯度",
                    },
                    "longitude": {
                        "type": "string",
                        "description": "経度",
                    },
                },
                "required": ["latitude", "longitude"],
            },
        },
    }
]

# LLM

In [5]:
def llm_agent(user_input):

    client = OpenAI()

    messages = [
        {"role": "system", "content": "You are a helpful assistant. Use the supplied tools to assist the user if needed."},
        {"role": "user", "content": user_input}
    ]

    # ChatGPT API呼び出し
    response = client.chat.completions.create(
        model=model_name,
        messages=messages,
        tools=tools,
        tool_choice='auto',
    )

    # ツール呼び出しの結果があるか確認
    tool_calls = getattr(response.choices[0].message, 'tool_calls', None)  # tool_callsが存在しない場合はNone

    if tool_calls:
        for tool_call in tool_calls:
            function_name = tool_call.function.name  # 属性としてアクセス
            arguments = json.loads(tool_call.function.arguments)

            # 関数の種類に基づいて処理を呼び出す
            if function_name == 'get_stock_price_range':
                function_response = get_stock_price_range(arguments)
            elif function_name == 'get_weather_by_location':
                function_response = get_weather_by_location(arguments)
            else:
                raise NotImplementedError(f"Unknown function: {function_name}")

            # ツール呼び出しごとに適切に応答を作成
            second_response = client.chat.completions.create(
                model=model_name,
                messages=[
                    {'role': 'system', 'content': "You are a helpful assistant."},
                    {"role": "user", "content": user_input},
                    {"role": "assistant", "tool_call_id": tool_call.id, "content": ""},  # 空文字列を設定
                    {
                        'role': 'function',
                        'name': function_name,
                        'content': str(function_response),
                    },
                ]
            )
            # 最終的な応答を表示
            print(second_response.choices[0].message.content)
    else:
        # 関数が呼び出されていない場合、通常の応答を表示
        print(response.choices[0].message.content)


In [6]:
llm_agent('三井住友フィナンシャルグループの2024年8月の株価の動きと、9月の見通しについて分析してください。')

三井住友フィナンシャルグループ（SMFG）の2024年8月の株価の動きを分析すると、以下のようなトレンドが見られます。

### 2024年8月の株価動向

- **月初（8月1日）**: 10880円からスタートし、高値を記録しました。
- **中旬（8月15日頃）**: 9699円に下がり、そこから急落が見られました。
- **下旬（8月28日頃）**: 9604円まで回復するも、最終的には9550円で取引を終えました。
- **全体の傾向**: この月は8月1日から中旬にかけての高値と、15日以降の急激な下落が特徴的でした。特に、8月8日の8647円という安値が特筆されます。その後の回復は見られましたが、8月の終わりは高値からかなりの下落がありました。

### 9月の見通し

- **テクニカル分析**: 株価が大きく動いた後の動きは、特にサポートラインがどこにあるかがポイントになります。9550円の水準がサポートとして機能するかどうかが注目されます。
- **市場環境**: 米国の金利動向や国際情勢（特にアジア市場との関連）など、外的要因が影響する可能性があります。特に金融株にとっては金利が重要ですので、今後の金融政策の動向には注意が必要です。
- **企業の業績と展望**: もしも三井住友フィナンシャルグループが良好な業績を発表するようであれば、9500円台からの回復が見込めるかもしれません。一方で、経済指標の悪化や予想を下回る決算があれば再度の売り圧力が強まる可能性があります。

### 結論

9月における三井住友フィナンシャルグループの株価は、9550円付近を中心に上下する可能性が高いと考えられます。サポートラインを維持できれば回復の兆しが見える一方、下方向への圧力が強まるようであればさらなる下落も考慮する必要があります。市場の動向や外的要因をしっかりと観察しつつ、投資判断を行うことが重要です。
三井住友フィナンシャルグループの2024年9月中旬までの株価の動きから分析を行います。

### 2024年8月の株価の動き
2024年8月の具体的な株価データが提供されていないため、ここでは2024年9月の初旬から中旬での株価の変動を見て、その背景を考察します。

### 2024年9月の初旬株価動向
- **9月2日**: 9597円
- **9月3日

In [7]:
llm_agent('東京の天気を教えてください。')

現在の東京の天気は、気温25.8℃、風速3.3m/s（風向き311°）です。天候は晴れています。昼間ではなく、夜の時間帯です。何か他に知りたいことがあれば教えてください！


In [8]:
llm_agent('LLMについて説明して下さい。')

LLMとは「大規模言語モデル（Large Language Model）」の略で、自然言語処理（NLP）分野で使用される人工知能（AI）の一種です。LLMは膨大な量のテキストデータを用いて訓練され、言語の構造や文脈を理解し、自然な言語生成を行う能力を持っています。

以下はLLMの特徴と機能についての詳細です。

1. **大量のデータで訓練**: LLMは数ギガバイトからテラバイト規模のテキストデータを使用して訓練されます。これにより、さまざまな文脈やトピックについて理解する能力が向上します。

2. **自己教師あり学習**: LLMは自己教師あり学習の手法を使用してリアルな文脈を学習します。例えば、次の単語を予測する作業を通じて、言葉の関連性や文の構造を学びます。

3. **自然言語生成**: 訓練されたLLMは、人間が自然に書いたようなテキストを生成することができます。これには、文章の作成、質問への応答、要約などが含まれます。

4. **多用途性**: LLMはさまざまなタスクに応用できます。例えば、翻訳、対話システム、情報検索、文書要約、文法チェックなどがあります。

5. **コンテキスト理解**: LLMは文脈を考慮に入れて、特定の状況や意図に基づいた応答を生成します。これにより、より人間らしい応答が可能になります。

6. **フィードバックループ**: 実際の使用から学習し、継続的にパフォーマンスを改善するためのフィードバックループを構築することも可能です。

代表的なLLMには、OpenAIのGPTシリーズ、GoogleのBERTやT5、MetaのLLaMAなどがあります。これらのモデルは、商業利用や研究の分野で広く活用されています。
